In [23]:
import requests
import json
import time
import pandas as pd
from datetime import datetime
from typing import Tuple, List, Optional
from etherscan_functions import get_erc20_transfers, get_block_number_by_timestamp

api_key = "VQAIR728IM4Z8RZKPYBR4ESM5I3WBZK2C1" # my free API key, you can get one at https://etherscan.io/myapikey
base_url = "https://api.etherscan.io/v2/api" # We're using the v2 API 2024/12/12
ADDRESS = "0x5be9a4959308A0D0c7bC0870E319314d8D957dBB" # Address of the contract we want to get the source code of
chain_id = 1  # Ethereum Mainnet

In [24]:
def get_block_numbers(
    start_date: str,
    end_date: str,
    closest: str = "before",
    include_all: bool = False,
) -> List[int]:
    """
    Returns block numbers for a specified date range.

    :param start_date: Start date in 'YYYY-MM-DD' format.
    :param end_date: End date in 'YYYY-MM-DD' format.
    :param closest: 'before' or 'after', to find the closest block.
    :param include_all: If True, returns all block numbers in the range. 
                        If False, only returns the start and end block numbers.
    :return: A list of block numbers. 
             If include_all is True, includes all blocks in the range. 
             Otherwise, includes only the start and end block numbers.

    :raises ValueError: If input dates are invalid or API responses indicate failure.
    :raises ConnectionError: If the HTTP request fails.
    """
    # Convert dates to Unix timestamps
    try:
        start_timestamp = int(datetime.strptime(start_date, "%Y-%m-%d").timestamp())
        end_timestamp = int(datetime.strptime(end_date, "%Y-%m-%d").timestamp())
    except ValueError:
        raise ValueError("Dates must be in 'YYYY-MM-DD' format.")

    if start_timestamp >= end_timestamp:
        raise ValueError("Start date must be earlier than end date.")
    
    # Get block numbers for the start and end timestamps
    start_block = get_block_number_by_timestamp(timestamp=start_timestamp, closest=closest)
    end_block = get_block_number_by_timestamp(timestamp=end_timestamp, closest=closest)

    # If include_all is True, generate the list of all blocks in the range
    if include_all:
        return list(range(start_block, end_block + 1))
    
    # Otherwise, return just the start and end blocks
    return [start_block, end_block]

In [26]:
# start_block, end_block = get_block_numbers(start_date="2024-11-10", end_date="2024-11-16", closest="before", include_all=False)
# set 'before' to get the block number less or qual to the timestamp. set 'after' to get the block number greater or equal to the timestamp

start_block, _ = get_block_numbers(start_date="2024-11-10", end_date="2024-11-16", closest="before", include_all=False)
_, end_block = get_block_numbers(start_date="2024-11-10", end_date="2024-11-16", closest="after", include_all=False)

transfers = get_erc20_transfers(
        address=ADDRESS,
        contract_address=None,
        startblock=start_block,
        endblock=end_block,
        page=1,
        offset=100,
    )
transfers['dateTime'] = pd.to_datetime(pd.to_numeric(transfers['timeStamp'], errors='coerce'), unit='s').dt.strftime('%Y-%m-%d %H:%M:%S')
# Reorder columns, moving 'dateTime' to the front
cols = ['dateTime'] + [col for col in transfers if col != 'dateTime']
transfers = transfers[cols]

transfers.describe()
transfers.to_csv('./erc20_transfers.csv', index=False)
